In [ ]:
# importacion general de librerias y de visualizacion (matplotlib y seaborn)
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

plt.style.use('default') # haciendo los graficos un poco mas bonitos en matplotlib
#plt.rcParams['figure.figsize'] = (20, 10)

sns.set(style="whitegrid") # seteando tipo de grid en seaborn

pd.options.display.float_format = '{:20,.2f}'.format # suprimimos la notacion cientifica en los outputs

import warnings
warnings.filterwarnings('ignore')

In [ ]:
columnas_categoricas = ['land_surface_condition', 'foundation_type', 'roof_type', 'ground_floor_type', 'other_floor_type', 'position', 'plan_configuration', 'legal_ownership_status']

# Binary Encoding

In [ ]:
!pip install category_encoders
import category_encoders as ce

In [ ]:
train_values_categoricos = train_values[columnas_categoricas]
encoder = ce.BinaryEncoder(cols=columnas_categoricas)
train_values_categoricos_ = train_values_categoricos.copy()

In [ ]:
df_bianrio = encoder.fit_transform(train_values_categoricos_)
df_bianrio

In [ ]:
train_values_sin_categoricas_ = train_values.drop(columns=columnas_categoricas)
train_values_sin_categoricas_[df_bianrio.columns] = df_bianrio
train_values_sin_categoricas_

# Mean Smoothing Encoding

In [ ]:
def encode_target_smooth(data, target, categ_variables, smooth):
    """    
    Apply target encoding with smoothing.
    
    Parameters
    ----------
    data: pd.DataFrame
    target: str, dependent variable
    categ_variables: list of str, variables to encode
    smooth: int, number of observations to weigh global average with
    
    Returns
    --------
    encoded_dataset: pd.DataFrame
    code_map: dict, mapping to be used on validation/test datasets 
    defaul_map: dict, mapping to replace previously unseen values with
    """
    train_target = data.copy()
    code_map = dict()   # stores mapping between original and encoded values
    default_map = dict()# stores global average of each variable
    
    for v in categ_variables:
        prior = data[target].mean()
        n = data.groupby(v).size()
        mu = data.groupby(v)[target].mean()
        mu_smoothed = (n * mu + smooth * prior) / (n + smooth)
        
        train_target.loc[:, v] = train_target[v].map(mu_smoothed)        
        code_map[v] = mu_smoothed
        default_map[v] = prior        
    return train_target, code_map, default_map

In [ ]:
train_target_smooth, target_map, default_map = encode_target_smooth(train_values.join(train_labels), 'damage_grade', columnas_categoricas, 100)

In [ ]:
train_smooth = train_target_smooth.drop(columns='damage_grade')
train_smooth

In [ ]:
train_values_target = train_values.join(train_labels)
train_values_target